In [66]:
import gym
import numpy as np

import time

In [67]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [68]:
import matplotlib.pyplot as plt

In [69]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable
if find_executable('latex'):
    plt.rc('text', usetex=True)

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

In [70]:
import torch

In [71]:
%reload_ext autoreload
%autoreload 2

In [72]:
import models.rnn as rnns
import models.mlp as mlps
import models.linear as linears
import control.agents as agents
import control.environments as env

In [73]:
from utils.notifications import Slack

In [74]:
import copy

# Setup

In [80]:
env_name = 'Taxi-v2'
#env_name = 'Breakout-ram-v0'

In [81]:
environment = env.Environment(
    environment=gym.make(env_name), 
    agent=None,
    verbose=True,
    max_steps=200,
    capacity=100,
    representation_method='one_hot_encoding'
)

In [82]:
model = linears.Linear(
    input_dimension=environment.get_input_dimension(), 
    n_actions=environment.n_actions,
)
optimiser = torch.optim.SGD(model.parameters(), lr=100, momentum=0)
#optimiser = torch.optim.Adam(model.parameters(), lr=1)
agent = agents.DQNAgent(model, optimiser, gamma=1., temperature=10, algorithm='sarsa', n_actions=environment.n_actions)
environment.agent = agent

In [18]:
print(environment.agent.q(environment.state_representation(environment.environment.reset())))
model.load_state_dict(torch.load('../saved/taxi/mlp/state_dict.pth'))
agent.commit()
print(environment.agent.q(environment.state_representation(environment.environment.reset())))
print(environment.boltzmann(environment.state_representation(environment.environment.reset())))

[-152.5814  -151.62679 -152.77357 -152.7423  -151.98988 -152.57753]
[-152.5814  -151.62679 -152.77357 -152.7423  -151.98988 -152.57753]
[0.123688   0.32130238 0.10206328 0.10530633 0.22347164 0.12416831]


# Experiment

## Boltzmann

In [83]:
q_estimation = []
returns = []

iterator = environment.tqdm(range(10), ascii=True, ncols=100)

with iterator as it:
    for _ in it:

        environment.agent.commit()
        returns.append(environment.exploration_segment(100))

        for _ in range(len(environment.replay_memory) // 100):
            environment.batch(100)


  0%|                                                                        | 0/10 [00:00<?, ?it/s]

tensor([ 0.0179,  0.0171, -0.0369,  0.0416, -0.0014,  0.0317],
       grad_fn=<SqueezeBackward3>)
[-0.96207259 -1.03343709 -0.99063242 -1.03538522 -0.97150672 -0.99074415]
2
tensor([ 0.0179,  0.0171, -0.9973,  0.0416, -0.0014,  0.0317],
       grad_fn=<SqueezeBackward3>)
[-0.97159221 -0.99026757 -0.98455094 -1.0332068  -1.04196193 -0.99212728]
2
tensor([ 0.0179,  0.0171, -1.0023,  0.0416, -0.0014,  0.0317],
       grad_fn=<SqueezeBackward3>)
[ -9.96996519 -10.01584831 -10.0068027  -10.0414812   -9.97307436
 -10.03748195]
4
tensor([ 0.0179,  0.0171, -1.0023,  0.0416, -1.0014,  0.0317],
       grad_fn=<SqueezeBackward3>)
[ -9.97640599 -10.01985015  -9.96128758  -9.96761143  -9.98607564
  -9.99611175]
5
tensor([ 0.0179,  0.0171, -1.0023,  0.0416, -1.0014, -0.9683],
       grad_fn=<SqueezeBackward3>)
[-0.9829094  -0.99245035 -1.014271   -0.98509856 -1.03267581 -1.03107423]
3
tensor([ 0.0179,  0.0171, -1.0023, -0.9584, -1.0014, -0.9683],
       grad_fn=<SqueezeBackward3>)
[ -9.9829094   -9.

 10%|######4                                                         | 1/10 [00:03<00:27,  3.10s/it]


0
tensor([-1.0033, -0.9737, -0.9981, -0.9814, -9.9932, -9.9938],
       grad_fn=<SqueezeBackward3>)
[-0.96207259 -1.03343709 -0.99063242 -1.03538522 -0.97150672 -0.99074415]
0


 20%|############8                                                   | 2/10 [00:05<00:23,  2.97s/it]

...

tensor([ -7.0635,  -6.4738,  -6.2872,  -4.8983, -14.3513, -15.4462],
       grad_fn=<SqueezeBackward3>)
[ -7.73499918  -7.95159101  -5.35261631  -7.76408339 -13.02970982
  -9.9946022 ]
3
tensor([ -7.0635,  -6.4738,  -6.2872,  -5.8073, -14.3513, -15.4462],
       grad_fn=<SqueezeBackward3>)
[-15.65279007 -16.47212458 -17.12094212 -15.87608433 -23.08955479
 -22.95127869]
5
tensor([ -7.0635,  -6.4738,  -6.2872,  -5.8073, -14.3513, -16.2189],
       grad_fn=<SqueezeBackward3>)
[ -6.60701752  -7.67869282  -7.34293413  -7.70548964 -14.76137257
 -16.68464375]
0
tensor([ -7.5769,  -6.4738,  -6.2872,  -5.8073, -14.3513, -16.2189],
       grad_fn=<SqueezeBackward3>)
[ -7.30608368  -7.14953899  -7.23261309  -7.97120619 -14.02409077
 -10.00884247]
1
tensor([ -7.5769,  -7.3516,  -6.2872,  -5.8073, -14.3513, -16.2189],
       grad_fn=<SqueezeBackward3>)
[-15.65279007 -16.47212458 -17.12094212 -15.87608433 -23.08955479
 -22.95127869]
5
tensor([ -7.5769,  -7.3516,  -6.2872,  -5.8073, -14.3513, -16.593

 90%|#########################################################6      | 9/10 [00:22<00:02,  2.45s/it]

tensor([ -7.7080,  -7.8237,  -7.4823,  -7.8274, -14.3513, -16.7533],
       grad_fn=<SqueezeBackward3>)
[ -8.43062735  -9.09108639  -8.92885923  -8.54824877 -17.14206886
 -13.34205818]
3
tensor([ -7.7080,  -7.8237,  -7.4823,  -8.7147, -14.3513, -16.7533],
       grad_fn=<SqueezeBackward3>)
[ -9.07594299  -8.56095362  -9.00840282  -8.99640083 -17.4834919
 -17.55815315]
1
tensor([ -7.7080,  -8.7799,  -7.4823,  -8.7147, -14.3513, -16.7533],
       grad_fn=<SqueezeBackward3>)
[ -8.3654933   -8.91656399  -7.88962841  -8.63383722 -16.9597578
 -11.82638645]
2
tensor([ -7.7080,  -8.7799,  -8.3641,  -8.7147, -14.3513, -16.7533],
       grad_fn=<SqueezeBackward3>)
[ -8.00485849  -8.20733118  -7.08977842  -8.50148821 -15.27268696
 -13.99038887]
1
tensor([ -7.7080,  -8.6756,  -8.3641,  -8.7147, -14.3513, -16.7533],
       grad_fn=<SqueezeBackward3>)
[-17.89432573 -17.78849459 -17.39315033 -17.87327194 -27.09758949
 -26.55487442]
4
tensor([ -7.7080,  -8.6756,  -8.3641,  -8.7147, -15.3513, -16.7533]

100%|###############################################################| 10/10 [00:25<00:00,  2.52s/it]

tensor([ -8.8999,  -8.9361,  -9.2072,  -9.1079, -16.3365, -17.3209],
       grad_fn=<SqueezeBackward3>)
[ -7.86847067  -9.25383091  -9.58302498 -10.03838539 -14.60417652
 -17.02233887]
1
tensor([ -8.8999,  -9.4302,  -9.2072,  -9.1079, -16.3365, -17.3209],
       grad_fn=<SqueezeBackward3>)
[ -9.7943697   -7.09196615  -7.83079863  -7.38142252  -6.25300121
 -18.52843094]
3
tensor([ -8.8999,  -9.4302,  -9.2072,  -8.7223, -16.3365, -17.3209],
       grad_fn=<SqueezeBackward3>)
[ -9.72178555  -8.88359976  -7.68906498  -7.99149656 -13.69407654
 -15.49077892]
2
tensor([ -8.8999,  -9.4302,  -9.2390,  -8.7223, -16.3365, -17.3209],
       grad_fn=<SqueezeBackward3>)
[ -8.75641203  -8.92146826  -6.7844348   -8.28318262 -15.66818142
 -17.7232933 ]
0
tensor([ -8.9435,  -9.4302,  -9.2390,  -8.7223, -16.3365, -17.3209],
       grad_fn=<SqueezeBackward3>)
[ -9.70413589  -9.06889057 -10.17998886  -9.7267704  -14.95728683
 -18.40656853]
0
tensor([ -9.7217,  -9.4302,  -9.2390,  -8.7223, -16.3365, -17.320

In [36]:
environment.reset()

done = False
full_return = 0.

counter = 0
while not done and counter < environment.max_steps:
    
    s, reward, done, i = environment.step(environment.action)

    p, q = environment.boltzmann(s, return_q=True)
    a = environment.sample_action(p)
    
    environment.state, environment.action = s, a

    full_return = environment.agent.gamma * full_return + reward
    counter += 1
    
    print(np.argmax(s))
    print(environment.agent.q(s))


329
[-30.709248 -31.531038 -30.673918 -31.595798 -32.558308 -31.055958]
429
[-31.233753 -30.915483 -18.555916 -29.451986 -35.169575 -38.4846  ]
449
[-31.233753 -30.93541  -30.999357 -32.537136 -32.77127  -38.474052]
349
[-30.96778  -30.647896 -28.851017 -13.912269 -29.423548 -28.960613]
329
[-30.709248 -31.531038 -30.673918 -31.595798 -32.558308 -31.055958]
429
[-31.233753 -30.915483 -18.555916 -29.451986 -35.169575 -38.4846  ]
449
[-31.233753 -30.93541  -30.999357 -32.537136 -32.77127  -38.474052]
349
[-30.96778  -30.647896 -28.851017 -13.912269 -29.423548 -28.960613]
329
[-30.709248 -31.531038 -30.673918 -31.595798 -32.558308 -31.055958]
349
[-30.96778  -30.647896 -28.851017 -13.912269 -29.423548 -28.960613]
329
[-30.709248 -31.531038 -30.673918 -31.595798 -32.558308 -31.055958]
429
[-31.233753 -30.915483 -18.555916 -29.451986 -35.169575 -38.4846  ]
449
[-31.233753 -30.93541  -30.999357 -32.537136 -32.77127  -38.474052]
449
[-31.233753 -30.93541  -30.999357 -32.537136 -32.77127  -38.

In [26]:
n_episodes = 3
agent.temperature = 0.1

plt.figure()

for i in range(n_episodes):

    full_return, counter, observations = environment.evaluation_episode(render=False,return_observations=True)
    
    q = []
    
    observation_old = None
    q_old = None
    
    for observation in observations:
        
        observation_new = environment.state_representation(observation)
        q_new = environment.agent.q(environment.state_representation(observation))
        
        if observation_old is not None:
            same_obs = np.array_equal(observation_old, observation_new)
            same_q = np.array_equal(q_old, q_new)
            if not same_obs and not same_q:
                print("great")
        
        observation_old = observation_new
        q_old = q_new
        
        q.append(q_new)
        
    q = np.asarray(q)
    print(np.std(q, axis=0))
    print(q[:,0])
    plt.plot(q[:,0], label='0')
    break
    #plt.plot(q[:,1], label='1')
    #plt.plot(q[:,2], label='2')
    
plt.show()

AttributeError: 'TaxiEnv' object has no attribute '_get_obs'

<Figure size 432x288 with 0 Axes>

In [15]:
plt.figure()
for i in range(n_episodes):
    x = np.asarray(q_estimation[i])
    plt.plot(x[:,1])

plt.show()

NameError: name 'q_estimation' is not defined

<Figure size 432x288 with 0 Axes>

## Testing

In [55]:
agent.temperature = 100
for _ in range(1):
    environment.exploration_episode(render=True)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
191
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
191
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
291
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
291
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
391
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
391
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
391
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
391
+---------+
|R: | : :G|
| : : : : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
431
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
431
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
431
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
431
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
431
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
451
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
451
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
451
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
451
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
351
+---------+
|R: 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
191
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
191
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
191
+---------+
|R: | : :

In [50]:
for _ in range(5):
    environment.evaluation_episode(render=True)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| |

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
92
+---------+
|R: | : :G|
| : : : : |
| : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
91
+---------+
|R: | : :G|
| : : 

...

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
212
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
112
+---------+
|R: | : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : 

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
149
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
249
+---------+
|R: | : 